In [ ]:
from sklearn.model_selection import StratifiedKFold,KFold,GroupKFold
import warnings
warnings.filterwarnings('ignore')
import time

from catboost import CatBoostClassifier
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder ,OneHotEncoder
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix ,precision_recall_curve
from sklearn.metrics import roc_auc_score
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',100)


In [ ]:
train=pd.read_csv('\\Users\\91993\\Desktop\\data\\train.csv_VsW9EGx\\train.csv')
test=pd.read_csv('\\Users\\91993\\Desktop\\data\\test.csv_yAFwdy2\\test.csv')
sample=pd.read_csv('\\Users\\91993\\Desktop\\data\\sample_submission_iA3afxn.csv')

In [ ]:
output=np.array(train['Response'].value_counts())

In [ ]:
GenderMap={
    'Male':0,
    'Female':1
}

vehicleAgeMap={
    '< 1 Year':1,
    '1-2 Year':2,
    '> 2 Years':4,
}

vehicleDamageMap={
    'Yes':2,
    'No':1
}

In [ ]:
y=train['Response']
train.drop(columns=['Response'],inplace=True)
train['train_or_test']='train'
test['train_or_test']='test'
df=pd.concat([train,test])
df.drop(columns=['id'],inplace=True)

In [ ]:
df["Region_Code"]= df["Region_Code"].astype(int) 
df["Policy_Sales_Channel"]= df["Policy_Sales_Channel"].astype(int)
df["Vintage"]= df["Vintage"].astype(float)
df["Age"]= df["Age"].astype(float) 

In [ ]:
cols=['Gender', 'Driving_License', 'Region_Code','Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage']
df['amount']=df.groupby(cols)['Annual_Premium'].transform('mean')
#df['unique_policy']=df.groupby(cols)['Policy_Sales_Channel'].transform('nunique')
#df['unique_policy']=df['unique_policy'].astype(float)
#df['per_amount']=df.groupby(cols)['Vintage'].transform('mean')

In [ ]:
df['Vehicle_Age']=df['Vehicle_Age'].map(vehicleAgeMap)
df['Gender']=df['Gender'].map(GenderMap)
df['Vehicle_Damage']=df['Vehicle_Damage'].map(vehicleDamageMap)

In [ ]:
#df['damage to Age Ratio']=df['Vehicle_Damage']/df['Vehicle_Age']

In [ ]:
df['total_policy_in_region']=df.groupby(['Region_Code'])['Policy_Sales_Channel'].transform('nunique')
df['total_region_covered_by_policy']=df.groupby(['Policy_Sales_Channel'])['Region_Code'].transform('nunique')
df['total_policy_in_region']=df['total_policy_in_region'].astype(float)
df['total_region_covered_by_policy']=df['total_region_covered_by_policy'].astype(float)
df['Amount_per_policy_channel']=df.groupby(['Policy_Sales_Channel'])['Annual_Premium'].transform('mean')

In [ ]:
train=df.loc[df.train_or_test.isin(['train'])]
test=df.loc[df.train_or_test.isin(['test'])]
train.drop(columns={'train_or_test'},axis=1,inplace=True)
test.drop(columns={'train_or_test'},axis=1,inplace=True)

In [ ]:
output[0]/output[1]

In [ ]:
categorical_features_indices = np.where(train.dtypes != np.float)[0]

In [ ]:
categorical_features_indices

In [ ]:
import time
from sklearn.model_selection import StratifiedKFold,RepeatedStratifiedKFold
skf = StratifiedKFold(n_splits=10,shuffle=True,random_state=101)
#skf = RepeatedStratifiedKFold(n_splits=10,n_repeats=2,random_state=0)
cv_score = []
clf_list=[]
#pred_test =np.zeros((len(test_),11))

X=train
i=0
for train_index,test_index in skf.split(train,y):
    x_train,x_val = train.iloc[train_index,:],train.iloc[test_index,:]
    y_train,y_val = y[train_index],y[test_index]
    start = time.time()
    eval_set=[(x_val,y_val)]
    
    clf=CatBoostClassifier(n_estimators=1000,
                       random_state=0,
                       eval_metric='AUC',
                       learning_rate=0.08,
                       scale_pos_weight=4,
                       #task_type='GPU',
                       cat_features=categorical_features_indices
                       #num_leaves=64
                       )

    clf.fit(x_train,y_train,early_stopping_rounds=100,eval_set=eval_set, verbose=100)
     
    #clf = xgboost.XGBClassifier(gpu_id=0,tree_method='gpu_hist',max_depth=8,scale_pos_weight=output[0]/output[1],n_estimators=400,learning_rate=0.015)
    #clf=xgboost.XGBClassifier(max_depth=5,scale_pos_weight=output[0]/output[1],n_estimators=1000,learning_rate=0.05)
    #clf = xgboost.XGBClassifier(gpu_id=0,tree_method='gpu_hist',max_depth=8,min_child_weight=77,scale_pos_weight=output[0]/output[1],n_estimators=437,learning_rate=0.01875698662271597)
    #clf.fit(x_train,y_train,eval_set=eval_set,early_stopping_rounds=100,eval_metric='auc',verbose=100)
   
    elapsed = time.time() - start
    print("Parallel Thread XGBoost and CV: %f" % (elapsed))
    clf_list.append(clf)
    y_pred=clf.predict_proba(x_val)[:,1]
   # score = accuracy_score(y_val,clf.predict(x_val))
    score=roc_auc_score(y_val, y_pred)
    i=i+1
    print(i,score)
    cv_score.append(score)
    
print(np.mean(cv_score))

In [ ]:
pred_test =np.zeros(len(test))
for clf in clf_list:
    pred_test=pred_test+clf.predict_proba(test)[:,1]
    #pred_test=max_prob(pred_test,clf.predict_proba(df_test.values))
pred_test = pred_test/10

sample['Response']=pred_test
sample.to_csv('sample149.csv',index=False)